- 直接求解$Ax = b $,复杂度：$O(n^{3})
- 求解上三角或下三角矩阵，复杂度：$O(n^{2})
# LU分解
$$Ax = b \Rightarrow LUx = b  ( 令Ux = y ) \Rightarrow Ly = b $$
- 原本的$Ax = b$ 的计算复杂度是$O(n^{3})$
- 经过这一些列变换，计算复杂度是一个$O(n^{3})$[高斯消元法，化上三角] 和2个$O(n^{2})$ [上三角求解]

A: symmetric positive definite(SPD,对称正定),可以用$LL^{T}$

对于三对角矩阵，追赶法

In [1]:
using BenchmarkTools

In [2]:
function back_sub(A, b)
    x = zeros(n)
    for i = n:-1:1
        x[i] = b[i]
        for j = i + 1 : n
            x[i] -= A[i, j] * x[j]
        end
        x[i] = x[i] / A[i, i]
    end
    return x
end

back_sub (generic function with 1 method)

In [1]:
function forward_sub(A,b)
    x = zeros(n)
    for i = 1:n
        x[i] = b[i]
        # println("x [$i] =" , x[i])
        for j = 1 : i-1
            x[i] -= A[i, j] * x[j]
        end
        # println("i = ",i)
        x[i] = x[i] / A[i, i]
        # println("最后的x [$i] =" , x[i])
    end
    return x
end

forward_sub (generic function with 1 method)

In [6]:
function LU_Factorization(A,b)
    n = size(A, 1)
    L = zeros(n, n)
    U = copy(A)

    for k = 1:n
        for i = k + 1 : n
            L[i,k] = U[i,k]/U[k,k]
            for j = k : n
                U[i, j] -= L[i,k] * U[k,j]
            end
        end
    end

    for i = 1:n
        L[i,i] = 1
    end
    # println(L)
    # println(U)
    # return L,U
    y = forward_sub(L,b)
    x = back_sub(U, y)
    return x
end

LU_Factorization (generic function with 2 methods)

In [61]:
n = 4;
A = [1.0 1.0 2.0 3.0;0.0 2.0 1.0 2.0;1.0 -1.0 2.0 2.0;2.0 2.0 5.0 9.0];
b = [3.0; 1.0; 3.0; 7.0];
L,U = LU_Factorization(A)
y = forward_sub(L,b)
x = back_sub(U, y)

4-element Vector{Float64}:
 1.0
 0.0
 1.0
 0.0

In [10]:
n = 3

A = zeros(n, n)
for i = 1:n
    for j = 1:n
        A[i, j] = randn(1)[1]
    end
end
b = randn(n);
LU_Factorization(A,b)

[1.0 0.0 0.0; 0.6532806800943114 1.0 0.0; -0.44007059853786257 -0.12259446317460458 1.0]
[-1.6684711334307656 -0.5001572510167872 -0.24927706894239046; 0.0 2.258829437896355 1.4392565812898983; 0.0 0.0 1.576859851597865]


In [63]:
n = 3;
A = [2.0 0.0 0.0;-1.0 3.0 0.0;1.0 -2.0 4.0];
b = [1.0; -1.0; 2.0];
L,U = LU_Factorization(A)
y = forward_sub(L,b)
x = back_sub(U, y)

3-element Vector{Float64}:
  0.5
 -0.16666666666666666
  0.2916666666666667

In [7]:
using PyPlot
#随机生成n阶矩阵
n = 100
A = zeros(n, n)
for i = 1:n
    for j = 1:n
        A[i, j] = randn(1)[1]
    end
end
b = randn(n);
# figure()
# spy(A)
# display(gcf())
L,U = LU_Factorization(A,b)
# y = forward_sub(L,b)
# x = back_sub(U, y)

100-element Vector{Float64}:
  2.226881149536307
  0.8249738736368465
  1.761677022781638
  0.4135329870174612
 -1.4793199775694028
 -2.032983654855264
  3.857994755667302
  1.149924810211032
  0.6364672675709054
  1.9446595024729085
  ⋮
  2.032966722209989
 -1.556791386253424
  2.181595936275801
 -0.6437097216701141
 -0.04337249357606358
  1.2770659538849933
  0.7761230123392369
  1.4598408272471608
 -1.8587684227466685

In [8]:
A\b

100-element Vector{Float64}:
  2.2268811495365006
  0.8249738736371229
  1.7616770227821594
  0.4135329870176712
 -1.479319977569343
 -2.0329836548562397
  3.857994755665964
  1.1499248102119903
  0.6364672675718483
  1.9446595024743378
  ⋮
  2.0329667222118646
 -1.5567913862532055
  2.1815959362769113
 -0.643709721670235
 -0.04337249357533035
  1.2770659538861486
  0.7761230123397053
  1.4598408272464247
 -1.8587684227463277

第四次作业
- 写一个lineSolver.jl文件（截图发邮箱）
- drawio.com，

In [3]:
#三对角矩阵求解（追赶法）
function tri_DiagSolver(A,b)
    n = size(A, 1)
    u = zeros(n)
    y = zeros(n)

    u[1] = A[1,2]/A[1,1]
    y[1] = b[1]/A[1,1]
    for i = 2:n-1
        u[i] = A[i,i+1]/(A[i,i]-u[i-1]*A[i,i-1])
        y[i] = ((b[i]-y[i-1]*A[i,i-1])/(A[i,i]-u[i-1]*A[i,i-1]))        
    end
    y[n] = ((b[n]-y[n-1]*A[n,n-1])/(A[n,n]-u[n-1]*A[n,n-1]))
    # 回代求解x
    x = zeros(n)
    x[n] = y[n]
    for i = n-1:-1:1
        x[i] = y[i] - u[i] * x[i+1]
    end
    return x
end

tri_DiagSolver (generic function with 1 method)

In [18]:
A = [1.0 2.0 0.0; 2.0 1.0 1.0; 0.0 1.0 1.0]
b = [1.0, 2.0, 3.0]
tri_DiagSolver(A,b)

3-element Vector{Float64}:
 -0.5
  0.75
  2.25

In [4]:
#随机生成三对角矩阵
using PyPlot

n = 100
A = zeros(n, n)
for i = 1:n
    A[i, i] = randn(1)[1]
    if i>1
        A[i, i-1] = randn(1)[1] 
    end
    if i<n
        A[i,i+1] = randn(1)[1]
    end
end
b = randn(n);
# println(A)
# figure()
# spy(A)
# display(gcf())
tri_DiagSolver(A,b)

100-element Vector{Float64}:
  0.12234197641369979
  0.880210707772036
 -4.891641931321264
  4.387269595426911
 -2.122108495939459
  0.7354856290979863
  2.8463428361344723
  0.5705672573644647
  0.19392040076033534
  0.40335068157077614
  ⋮
  0.5456428470288335
  1.472453358098277
  0.7916868660886545
  0.6737233001943483
 -2.450271497955893
  2.285364972070626
 -1.8655482137850008
 -0.14901642260997883
 -0.25402740606336344

In [5]:
A\b

100-element Vector{Float64}:
  0.12234197641369977
  0.8802107077720371
 -4.891641931321263
  4.387269595426911
 -2.122108495939459
  0.7354856290979868
  2.8463428361344723
  0.5705672573644649
  0.19392040076033534
  0.4033506815707763
  ⋮
  0.5456428470288331
  1.4724533580982775
  0.7916868660886541
  0.6737233001943483
 -2.450271497955893
  2.285364972070626
 -1.8655482137850008
 -0.14901642260997883
 -0.25402740606336344